In [1]:
import sqlite3
import pandas as pd

To use SQL in python we will need:
- A connection, or handshake with the database.
- A cursor object, to implement our queries.
- A query, our actual SQL code.

In [ ]:
Pathname = Our_Database_Object
sql_connect = sqlite3.connect(*Pathname)

In [ ]:
cursor = sql_connect.cursor()

In [ ]:
query = "SELECT * FROM;"

In [ ]:
results = cursor.execute(query).fetchall()

In [ ]:
pd.read_sql_query(query,sql_connect)

In [ ]:
sql_connect.close()

In [ ]:
#Outputs histograms from query

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
query = '''
#Query text goes here.
'''
pd.read_sql_query(query, sql_connect).hist(ax=ax)

A standard use for querying a database might be to construct a window or CTE using some aggregation method for later analysis in Python. Following are some standard windowing and CTE queries.

In [ ]:
#Sample Window Tables

##Total Values
SELECT TableKey, Value, 
#Summation over Value by TableKey
       SUM(Value) 
#Create the window and partitions
       OVER(PARTITION BY TableKey) AS TotalValue
FROM TableName



##Counting Rows
SELECT TableKey, Value, 
#Number of rows per partition
              COUNT(Value) 
#Create the window and partitions
       OVER(PARTITION BY TableKey ORDER BY TableKey) AS TotalValue
FROM TableName

##Similarly, FIRST_VALUE, LAST_VALUE, LEAD, and LAG may be applied as a windowing function but will naturally require ORDER BY.

SELECT TableKey, DateValue, 
#Specify the previous OrderDate in the window
       LAG(DateValue) 
#Over the window, partition by territory & order by order date
       OVER(PARTITION BY TableKey ORDER BY DateValue) AS PreviousDateValue,
#Specify the next DateValue in the window
       LEAD(DateValue) 
#Create the partitions and arrange the rows
       OVER(PARTITION BY TableKey ORDER BY DateValue) AS NextDateValue
FROM TableName



##Running totals


SELECT TableKey, DateValue,
#Create a running total
       SUM(Value) 
#Create the partitions and arrange the rows
       OVER(PARTITION BY TableKey ORDER BY DateValue) AS RunningTotal
FROM TableName

#Both standard deviation and mode can be used in windows.

##Create a CTE Called ModeValue which contains two columns

WITH ModeValue (Value, ValueFrequency)
AS
(
    SELECT Value, 
    ROW_NUMBER() 
    OVER(PARTITION BY Value ORDER BY Value) AS ValueFrequency
    FROM TabelName 
)

SELECT Value AS TableMode
FROM ModeValue
#Select the maximum UnitPriceFrequency from the CTE
WHERE ValueFrequency IN (SELECT MAX(ValueFrequency) From ModeValue)


In [ ]:
#The generic method for creating a CTE.

WITH CTEName (Col1, Col2)
AS
#Define the CTE query
(
#The two columns from the definition above
    SELECT Col1, Col2
    FROM TableName
)  


In [ ]:
#Returns all tablevalues that match the max ordered by table key.

SELECT *
FROM TableName AS Tb1
#JOIN and create the derived table
JOIN (SELECT TableKey, MAX(TableValue) AS MaxTableValue FROM TableName GROUP BY TabelKey) AS Tb2
#JOIN on TableValue equal to MaxTableValue
ON Tb1.TableValue = Tb2.MaxTableValue
#Join on TableKey
AND Tb1.TableKey = Tb2.TableKey


##Sample CTEs

#Returns all information for the record that matches the maximum table value.

#Create the CTE
WITH CTEName (MaxTableValue)
AS (SELECT MAX(TableValue) FROM TableName)

SELECT *
FROM TableName AS Tb1
#Join the CTE  
JOIN CTEName AS Tb2
ON Tb1.TableValue = Tb2.MaxTableValue
